## Migration Testing - Leonardo

#### Read in Currents feed

In [73]:
from dotenv import load_dotenv
from currentsapi import CurrentsAPI
import os
import json
import requests
import time
import openai
import random

In [62]:
load_dotenv()
currents_api_key = os.environ['currents_api_key']
leonardo_api_key = os.environ['leonardo_key']
openai.api_key = os.environ['openai_key']

In [6]:
api = CurrentsAPI(api_key=currents_api_key)

In [48]:
news_feed = api.search(country="", language="en", category="world")

#### Process into GPT prompt

In [60]:
titles = [x['title'] for x in news_feed['news']]

In [75]:
selected_snippet = random.choice(titles)

In [76]:
prompt_template = '''
Using the following snippet encased in ``, generate an image prompt of an imaginery human superhero or superheroine that is relevant to the snippet, be creating and detailed in the description of this character.
Also come up with a creative backstory in around 50 words on this hero's origin story and his/her main superpower, and include some elements of the original snippet in the backstory.
`Faces of hope: The freed hostages of Gaza`

'''  # noqa: E501

response_template = '''
Imaginary Superhero: Lumina Salvator

Description:
Lumina Salvator, draped in a radiant cloak, emanates a warm glow reflecting the faces of the freed hostages. Their eyes shine with compassion, symbolizing the transformative power of hope emerging from the shadows.

Backstory:
Sarah Al-Hassan, a photojournalist, harnessed the energy of hope captured in her lens. Empowered, she became Lumina Salvator, using the light of captured moments to inspire healing and unity.

Main Superpower:
Photonic Healing - Lumina Salvator can harness and project healing energy through captured images, bringing solace and empowerment to those who have endured trauma, embodying the resilience of the freed hostages in Gaza.
'''  # noqa: E501


In [77]:
completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                'role': 'system',
                'content': 'You are a creative character designer'
            },
            {
                'role': 'user',
                'content': prompt_template
            },
            {
                'role': 'assistant',
                'content': response_template
            },
            {
                'role': 'user',
                'content': f'Do the same for the following snippet: `{selected_snippet}`'  # noqa: E501
            }
        ]
    )

image_prompt = completion.choices[0].message.content

print(selected_snippet)
print(image_prompt)

Decision on North Dakota Wild Horses Expected Next Year
Imaginary Superheroine: Equinox Rider

Description:
Equinox Rider, adorned in a sleek, equestrian-inspired suit, embodies the untamed spirit of wild horses. Her long, flowing hair matches the deep chestnut color of her loyal equine companion, as they navigate the battleground of justice.

Backstory:
As a passionate conservationist, Emily Rivers discovered a mystical amulet while studying wild horses in North Dakota. Transformed into Equinox Rider, she vows to protect these majestic creatures and their natural habitats from harm and exploitation.

Main Superpower:
Equine Empathy - Equinox Rider possesses the unique ability to communicate with and understand wild horses, forging a powerful bond that allows her to summon their strength and agility. She fights to preserve the freedom and existence of the wild horses in North Dakota, taking the decision into her own hands for their well-being.



In [78]:
url = "https://cloud.leonardo.ai/api/rest/v1/generations"

prompt = '''Equinox Rider, adorned in a sleek, equestrian-inspired suit, embodies the untamed spirit of wild horses. Her long, flowing hair matches the deep chestnut color of her loyal equine companion, as they navigate the battleground of justice.'''

#### Call Leonardo AI

In [79]:

url = "https://cloud.leonardo.ai/api/rest/v1/generations"

payload = {
    "height": 512,
    "modelId": "1e60896f-3c26-4296-8ecc-53e2afecc132",
    "prompt": prompt,
    "width": 512,
    "alchemy": True,
    "highResolution": True,
    "nsfw": True,
    "num_images": 1,
    "photoReal": False,
    "presetStyle": "CINEMATIC",
    "expandedDomain": True
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)
for k in range(20):
    print(f'Sleeping...{k}')
    time.sleep(1)

{"sdGenerationJob":{"generationId":"3af9c073-aef5-464f-b2d8-5cac533ac3f4","apiCreditCost":24}}
Sleeping...0
Sleeping...1
Sleeping...2
Sleeping...3
Sleeping...4
Sleeping...5
Sleeping...6
Sleeping...7
Sleeping...8
Sleeping...9
Sleeping...10
Sleeping...11
Sleeping...12
Sleeping...13
Sleeping...14
Sleeping...15
Sleeping...16
Sleeping...17
Sleeping...18
Sleeping...19


In [80]:
gen_id = response.json()['sdGenerationJob']['generationId']

#### Grab the generated image from a CDN

In [81]:
url = f"https://cloud.leonardo.ai/api/rest/v1/generations/{gen_id}"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"generations_by_pk":{"generated_images":[{"url":"https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/3af9c073-aef5-464f-b2d8-5cac533ac3f4/Leonardo_Diffusion_XL_Equinox_Rider_adorned_in_a_sleek_equestr_0.jpg","nsfw":false,"id":"d8517cf2-40a9-4f4a-85ac-4311f9829151","likeCount":0,"generated_image_variation_generics":[]}],"modelId":"1e60896f-3c26-4296-8ecc-53e2afecc132","prompt":"Equinox Rider, adorned in a sleek, equestrian-inspired suit, embodies the untamed spirit of wild horses. Her long, flowing hair matches the deep chestnut color of her loyal equine companion, as they navigate the battleground of justice.","negativePrompt":"","imageHeight":512,"imageWidth":512,"inferenceSteps":30,"seed":354823168,"public":false,"scheduler":"EULER_DISCRETE","sdVersion":"SDXL_0_9","status":"COMPLETE","presetStyle":"CINEMATIC","initStrength":null,"guidanceScale":null,"id":"3af9c073-aef5-464f-b2d8-5cac533ac3f4","createdAt":"2023-11-25T12:08:39.116","promptMagic":false,"promp

In [88]:
response.json()['generations_by_pk']['generated_images'][0]['url']

'https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/3af9c073-aef5-464f-b2d8-5cac533ac3f4/Leonardo_Diffusion_XL_Equinox_Rider_adorned_in_a_sleek_equestr_0.jpg'

In [87]:
response.json()['generations_by_pk']

{'generated_images': [{'url': 'https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/3af9c073-aef5-464f-b2d8-5cac533ac3f4/Leonardo_Diffusion_XL_Equinox_Rider_adorned_in_a_sleek_equestr_0.jpg',
   'nsfw': False,
   'id': 'd8517cf2-40a9-4f4a-85ac-4311f9829151',
   'likeCount': 0,
   'generated_image_variation_generics': []}],
 'modelId': '1e60896f-3c26-4296-8ecc-53e2afecc132',
 'prompt': 'Equinox Rider, adorned in a sleek, equestrian-inspired suit, embodies the untamed spirit of wild horses. Her long, flowing hair matches the deep chestnut color of her loyal equine companion, as they navigate the battleground of justice.',
 'negativePrompt': '',
 'imageHeight': 512,
 'imageWidth': 512,
 'inferenceSteps': 30,
 'seed': 354823168,
 'public': False,
 'scheduler': 'EULER_DISCRETE',
 'sdVersion': 'SDXL_0_9',
 'status': 'COMPLETE',
 'presetStyle': 'CINEMATIC',
 'initStrength': None,
 'guidanceScale': None,
 'id': '3af9c073-aef5-464f-b2d8-5cac533ac3f4',
 'createdAt': '20